<a href="https://colab.research.google.com/github/ankit-genzeon/AI-ML-Bootcamp-Genzeon-2023/blob/master/Bot_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
Dataset = '/content/drive/MyDrive/Colab Notebooks/DataSet/bot_detection_data.csv'
df = pd.read_csv(Dataset)

In [ ]:
df.head()

In [ ]:
df.describe()

,User ID,Retweet Count,Mention Count,Follower Count,Bot Label
count,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000
mean,548890.680540,50.00560,2.513760,4988.602380,0.500360
std,259756.681425,29.18116,1.708563,2878.742898,0.500005
min,100025.000000,0.00000,0.000000,0.000000,0.000000
25%,323524.250000,25.00000,1.000000,2487.750000,0.000000
50%,548147.000000,50.00000,3.000000,4991.500000,1.000000
75%,772983.000000,75.00000,4.000000,7471.000000,1.000000
max,999995.000000,100.00000,5.000000,10000.000000,1.000000


In [ ]:
df.shape

(50000, 11)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize the relationship between Retweet Count and Follower Count
sns.scatterplot(data=df, x='Retweet Count', y='Follower Count')
plt.title('Relationship between Retweet Count and Follower Count')
plt.show()

# Count of Verified and Unverified Accounts
sns.countplot(data=df, x='Verified')
plt.title('Count of Verified and Unverified Accounts')
plt.xlabel('Verified')
plt.ylabel('Count')
plt.show()


In [ ]:
from sklearn.preprocessing import LabelEncoder
# Label encode the Verified column
label_encoder = LabelEncoder()
df['Verified'] = label_encoder.fit_transform(df['Verified'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Handling missing values
df['Hashtags'].fillna('Unknown', inplace=True)  # Replace missing locations with 'Unknown'

In [ ]:
#changing datatype of column 'Created At'

df['Created At']=pd.to_datetime(df['Created At'], format='%Y-%m-%d %H:%M')
print(df['Created At'].dtypes)

datetime64[ns]


In [ ]:
df.head()

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,0,1,Adkinston,2020-05-11 15:29:50,Unknown
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,1,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,1,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,1,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,0,1,Camachoville,2020-04-13 21:24:21,foreign mention


In [ ]:
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# Combine all text columns into a single Series
text_data = df['Tweet'] + ' ' + df['Username'] + ' ' + df['Hashtags']+' '+df['Location']

# Text vectorization using TF-IDF (sparse representation)
vectorizer = TfidfVectorizer()
text_sparse = vectorizer.fit_transform(text_data)

# Combine text features with additional features

additional_features = df[['Retweet Count', 'Verified','Mention Count','Follower Count','Created At']]

additional_features['Created At'] = additional_features['Created At'].astype(int)  # Convert to Unix timestamp
additional_features = additional_features.astype('float64')  # Convert to float64
text_sparse = text_sparse.astype('float64')

combined_sparse = hstack((text_sparse, additional_features))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_sparse, df['Bot Label'], test_size=0.2, random_state=42)

# Train the Random Forest classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)


<ipython-input-10-4b040857b371>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  additional_features['Created At'] = additional_features['Created At'].astype(int)  # Convert to Unix timestamp


RandomForestClassifier()

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create the bagging classifier
bagging_model = BaggingClassifier(rf_classifier, n_estimators=10, random_state=42)

# Train the bagging classifier
bagging_model.fit(X_train, y_train)

# Make predictions on the test set
predictions = bagging_model.predict(X_test)

# Calculate the accuracy of the bagging classifier
accuracy = accuracy_score(y_test, predictions)
print("Bagging accuracy:", accuracy)


Bagging accuracy: 0.4945


In [ ]:
df.head()

In [ ]:
# Make predictions on the testing data
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Accuracy: 0.4945
Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.54      0.51      4968
           1       0.50      0.45      0.48      5032

    accuracy                           0.49     10000
   macro avg       0.49      0.49      0.49     10000
weighted avg       0.49      0.49      0.49     10000



In [ ]:
# # Example: Predict labels for new data

# # New data (example)
# new_data = pd.DataFrame({
#     'Tweet': ['This is a new tweet'],
#     'Username': ['new_user'],
#     'Hashtags': ['#new'],
#     'Retweet Count': [5000],
#     'Verified': [1]
# })

# # Concatenate text features
# new_text_data = new_data['Tweet'] + ' ' + new_data['Username'] + ' ' + new_data['Hashtags']

# # Perform feature vectorization on new text features
# new_text_sparse = vectorizer.transform(new_text_data)

# # Combine new text features with additional features
# new_additional_features = new_data[['Retweet Count', 'Verified']]
# new_combined_sparse = hstack((new_text_sparse, new_additional_features))

# # Predict labels for the new data
# new_predictions = rf_classifier.predict(new_combined_sparse)

# # Print the predictions
# for i, prediction in enumerate(new_predictions):
#     if prediction == 1:
#         print(f"Data point {i+1}: Bot")
#     else:
#         print(f"Data point {i+1}: Not Bot")


Data point 1: Bot


In [ ]:
# pip install flask


In [ ]:
# from flask import Flask, jsonify, request
# from scipy.sparse import hstack
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.ensemble import RandomForestClassifier
# import pandas as pd

# # Load the trained model and vectorizer
# rf_classifier = RandomForestClassifier()
# vectorizer = TfidfVectorizer()

# # Load the preprocessed data (combined_sparse) if needed

# # Create the Flask app
# app = Flask(__name__)

# # Define a route for the prediction API
# @app.route('/predict', methods=['POST'])
# def predict():
#     # Get the JSON data from the request
#     data = request.get_json()

#     # Preprocess the data
#     text_data = data['Tweet'] + ' ' + data['Username'] + ' ' + data['Hashtags'] + ' ' + data['Location']
#     text_sparse = vectorizer.transform([text_data])

#     # Combine text features with additional features
#     additional_features = pd.DataFrame({
#         'Retweet Count': [data['Retweet Count']],
#         'Verified': [data['Verified']],
#         'Mention Count': [data['Mention Count']],
#         'Follower Count': [data['Follower Count']],
#         'Created At': [data['Created At']]
#     })

#     additional_features['Created At'] = additional_features['Created At'].astype(int)  # Convert to Unix timestamp
#     additional_features = additional_features.astype('float64')  # Convert to float64
#     text_sparse = text_sparse.astype('float64')

#     combined_sparse = hstack((text_sparse, additional_features))

#     # Predict the label
#     prediction = rf_classifier.predict(combined_sparse)[0]

#     # Return the prediction as JSON response
#     response = {'prediction': int(prediction)}
#     return jsonify(response)

# # Run the Flask app
# if __name__ == '__main__':
#     app.run(debug=True)
